In [6]:
%%js
import React, { useEffect, useMemo, useRef, useState } from "react";
import { Tabs, TabsList, TabsTrigger, TabsContent } from "@/components/ui/tabs";
import { Card, CardHeader, CardTitle, CardDescription, CardContent } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Slider } from "@/components/ui/slider";
import { Input } from "@/components/ui/input";
import { Sparkles, Orbit, Atom, Radar as RadarIcon, ChartScatter, LineChart as LineIcon, Rocket, Hexagon } from "lucide-react";
import * as d3 from "d3";
import * as THREE from "three";
import { ResponsiveContainer, LineChart, Line, AreaChart, Area, CartesianGrid, XAxis, YAxis, Tooltip, Legend, ScatterChart, Scatter, BarChart, Bar, RadarChart, PolarGrid, PolarAngleAxis, PolarRadiusAxis, Radar, PieChart, Pie, Cell } from "recharts";
// p5 is optional; if it's not available, we'll dynamically import in the component

// ---------- Helpers ----------
const tw = (...c) => c.filter(Boolean).join(" ");

function seededRandom(seed) {
  let t = seed % 2147483647;
  if (t <= 0) t += 2147483646;
  return () => (t = (t * 16807) % 2147483647) / 2147483647;
}

function linspace(start, end, n) {
  const step = (end - start) / (n - 1);
  return Array.from({ length: n }, (_, i) => start + i * step);
}

function pearsonR(xs, ys) {
  const n = xs.length;
  const meanX = d3.mean(xs);
  const meanY = d3.mean(ys);
  const num = d3.sum(xs.map((x, i) => (x - meanX) * (ys[i] - meanY)));
  const den = Math.sqrt(
    d3.sum(xs.map((x) => (x - meanX) ** 2)) * d3.sum(ys.map((y) => (y - meanY) ** 2))
  );
  return num / den;
}

function linearRegression(xs, ys) {
  const xMean = d3.mean(xs);
  const yMean = d3.mean(ys);
  const num = d3.sum(xs.map((x, i) => (x - xMean) * (ys[i] - yMean)));
  const den = d3.sum(xs.map((x) => (x - xMean) ** 2));
  const m = num / den;
  const b = yMean - m * xMean;
  return { m, b };
}

function kmeans(points, k = 3, maxIter = 50, seed = 42) {
  const rnd = seededRandom(seed);
  // init centers
  const centers = Array.from({ length: k }, () => points[Math.floor(rnd() * points.length)]);
  let labels = new Array(points.length).fill(0);
  for (let iter = 0; iter < maxIter; iter++) {
    // assign
    for (let i = 0; i < points.length; i++) {
      let best = 0;
      let bestDist = Infinity;
      for (let c = 0; c < k; c++) {
        const dx = points[i].x - centers[c].x;
        const dy = points[i].y - centers[c].y;
        const d = dx * dx + dy * dy;
        if (d < bestDist) {
          bestDist = d;
          best = c;
        }
      }
      labels[i] = best;
    }
    // recompute centers
    const sums = Array.from({ length: k }, () => ({ x: 0, y: 0, n: 0 }));
    for (let i = 0; i < points.length; i++) {
      const c = labels[i];
      sums[c].x += points[i].x;
      sums[c].y += points[i].y;
      sums[c].n += 1;
    }
    let changed = false;
    for (let c = 0; c < k; c++) {
      if (sums[c].n > 0) {
        const nx = sums[c].x / sums[c].n;
        const ny = sums[c].y / sums[c].n;
        if (Math.hypot(nx - centers[c].x, ny - centers[c].y) > 1e-6) {
          centers[c] = { x: nx, y: ny };
          changed = true;
        }
      }
    }
    if (!changed) break;
  }
  return { centers, labels };
}

function moons(n = 600, noise = 0.08, seed = 7) {
  const rnd = seededRandom(seed);
  const data = [];
  for (let i = 0; i < n; i++) {
    const moon = i < n / 2 ? 0 : 1;
    const angle = (Math.PI * (i % (n / 2))) / (n / 2);
    let x, y;
    if (moon === 0) {
      x = Math.cos(angle);
      y = Math.sin(angle);
    } else {
      x = 1 - Math.cos(angle);
      y = -Math.sin(angle) + 0.5;
    }
    x += (rnd() - 0.5) * noise * 2;
    y += (rnd() - 0.5) * noise * 2;
    data.push({ x, y, label: moon });
  }
  return data;
}

// ---------- Components ----------

function SectionHeader({ icon: Icon, title, desc }) {
  return (
    <div className="flex items-center gap-3 mb-4">
      <div className="p-2 rounded-2xl bg-gradient-to-br from-sky-500/30 to-cyan-400/30 shadow-inner">
        <Icon className="w-5 h-5" />
      </div>
      <div>
        <div className="text-xl font-semibold tracking-tight">{title}</div>
        {desc && <div className="text-sm text-muted-foreground">{desc}</div>}
      </div>
    </div>
  );
}

function TimeSeriesArcade() {
  const [points, setPoints] = useState(300);
  const [freq, setFreq] = useState(3);
  const data = useMemo(() => {
    const xs = linspace(0, 2 * Math.PI, points);
    return xs.map((x, i) => ({
      t: i,
      sin: Math.sin(x * freq) + Math.sin(x * (freq / 2)) * 0.4 + (Math.random() - 0.5) * 0.2,
      cos: Math.cos(x * (freq * 0.6)) + (Math.random() - 0.5) * 0.15,
    }));
  }, [points, freq]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={LineIcon} title="Time‑Series Arcade" desc="Sine, cosine, and controlled chaos" />
      </CardHeader>
      <CardContent className="space-y-4">
        <div className="grid grid-cols-1 md:grid-cols-3 gap-4 items-center">
          <div className="md:col-span-2 h-64">
            <ResponsiveContainer width="100%" height="100%">
              <AreaChart data={data}>
                <defs>
                  <linearGradient id="g1" x1="0" y1="0" x2="0" y2="1">
                    <stop offset="5%" stopOpacity={0.7} />
                    <stop offset="95%" stopOpacity={0} />
                  </linearGradient>
                </defs>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="t" />
                <YAxis />
                <Tooltip />
                <Legend />
                <Area type="monotone" dataKey="sin" strokeWidth={2} fillOpacity={0.25} fill="url(#g1)" />
                <Line type="monotone" dataKey="cos" strokeWidth={2} dot={false} />
              </AreaChart>
            </ResponsiveContainer>
          </div>
          <div className="space-y-4">
            <div>
              <div className="text-xs mb-2">Points: {points}</div>
              <Slider value={[points]} min={100} max={1000} step={50} onValueChange={(v) => setPoints(v[0])} />
            </div>
            <div>
              <div className="text-xs mb-2">Frequency: {freq}×</div>
              <Slider value={[freq]} min={1} max={12} step={1} onValueChange={(v) => setFreq(v[0])} />
            </div>
          </div>
        </div>
      </CardContent>
    </Card>
  );
}

function CorrelationLab() {
  const [rho, setRho] = useState(0.7);
  const [n, setN] = useState(200);
  const data = useMemo(() => {
    // Box-Muller for normal, then correlate via Cholesky
    const x = [];
    const y = [];
    for (let i = 0; i < n; i++) {
      const u1 = Math.random();
      const u2 = Math.random();
      const r = Math.sqrt(-2 * Math.log(u1));
      const th = 2 * Math.PI * u2;
      const z1 = r * Math.cos(th);
      const z2 = r * Math.sin(th);
      x.push(z1);
      y.push(rho * z1 + Math.sqrt(1 - rho * rho) * z2);
    }
    const reg = linearRegression(x, y);
    const line = linspace(-3, 3, 50).map((t, idx) => ({ x: t, y: reg.m * t + reg.b, id: idx }));
    return { pts: x.map((xi, i) => ({ x: xi, y: y[i], id: i })), r: pearsonR(x, y).toFixed(3), line };
  }, [rho, n]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={ChartScatter} title="Correlation Lab" desc="Dial the dependence & watch r change" />
      </CardHeader>
      <CardContent className="space-y-4">
        <div className="grid grid-cols-1 md:grid-cols-3 gap-4 items-center">
          <div className="md:col-span-2 h-64">
            <ResponsiveContainer width="100%" height="100%">
              <ScatterChart>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis type="number" dataKey="x" domain={["auto", "auto"]} />
                <YAxis type="number" dataKey="y" domain={["auto", "auto"]} />
                <Tooltip cursor={{ strokeDasharray: "3 3" }} />
                <Legend />
                <Scatter name="points" data={data.pts} />
                <Line type="monotone" dataKey="y" data={data.line} dot={false} />
              </ScatterChart>
            </ResponsiveContainer>
          </div>
          <div className="space-y-4">
            <div className="text-2xl font-semibold">r = {data.r}</div>
            <div>
              <div className="text-xs mb-2">Correlation (ρ)</div>
              <Slider value={[rho]} min={-0.95} max={0.95} step={0.05} onValueChange={(v) => setRho(parseFloat(v[0].toFixed(2)))} />
            </div>
            <div>
              <div className="text-xs mb-2">Points: {n}</div>
              <Slider value={[n]} min={50} max={1000} step={50} onValueChange={(v) => setN(v[0])} />
            </div>
          </div>
        </div>
      </CardContent>
    </Card>
  );
}

function ClusterStudio() {
  const [k, setK] = useState(3);
  const [noise, setNoise] = useState(0.08);
  const pts = useMemo(() => moons(600, noise), [noise]);
  const { labels, centers } = useMemo(() => kmeans(pts, k, 80), [pts, k]);
  const colors = ["#60a5fa", "#34d399", "#f472b6", "#f59e0b", "#a78bfa", "#f87171"]; // tailwind-ish palette
  const data = pts.map((p, i) => ({ ...p, c: labels[i], color: colors[labels[i] % colors.length] }));

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Atom} title="Cluster Studio (K‑Means)" desc="Moons, noise, and emergent structure" />
      </CardHeader>
      <CardContent className="space-y-4">
        <div className="grid grid-cols-1 md:grid-cols-3 gap-4 items-center">
          <div className="md:col-span-2 h-64">
            <ResponsiveContainer width="100%" height="100%">
              <ScatterChart>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis type="number" dataKey="x" domain={["auto", "auto"]} />
                <YAxis type="number" dataKey="y" domain={["auto", "auto"]} />
                <Tooltip />
                <Legend />
                {Array.from({ length: k }).map((_, ci) => (
                  <Scatter key={ci} name={`cluster ${ci + 1}`} data={data.filter((d) => d.c === ci)} fill={colors[ci % colors.length]} />
                ))}
              </ScatterChart>
            </ResponsiveContainer>
          </div>
          <div className="space-y-4">
            <div>
              <div className="text-xs mb-2">Clusters: {k}</div>
              <Slider value={[k]} min={2} max={6} step={1} onValueChange={(v) => setK(v[0])} />
            </div>
            <div>
              <div className="text-xs mb-2">Noise: {noise.toFixed(2)}</div>
              <Slider value={[noise]} min={0} max={0.3} step={0.01} onValueChange={(v) => setNoise(parseFloat(v[0].toFixed(2)))} />
            </div>
            <div className="text-xs text-muted-foreground">Centers: {centers.map((c) => `(${c.x.toFixed(2)}, ${c.y.toFixed(2)})`).join("  ")}</div>
          </div>
        </div>
      </CardContent>
    </Card>
  );
}

function ForceGraphPlayground() {
  const ref = useRef(null);
  const [nodes, setNodes] = useState(() => d3.range(25).map((i) => ({ id: i })));
  const [links, setLinks] = useState(() => d3.range(40).map(() => ({ source: Math.floor(Math.random() * 25), target: Math.floor(Math.random() * 25) })));

  useEffect(() => {
    const el = ref.current;
    if (!el) return;
    el.innerHTML = "";
    const width = el.clientWidth;
    const height = 320;
    const svg = d3
      .select(el)
      .append("svg")
      .attr("width", "100%")
      .attr("height", height)
      .attr("viewBox", `0 0 ${width} ${height}`);

    const sim = d3
      .forceSimulation(nodes)
      .force("link", d3.forceLink(links).id((d) => d.id).distance(50).strength(0.5))
      .force("charge", d3.forceManyBody().strength(-120))
      .force("center", d3.forceCenter(width / 2, height / 2));

    const link = svg
      .append("g")
      .attr("stroke", "currentColor")
      .attr("stroke-opacity", 0.3)
      .selectAll("line")
      .data(links)
      .join("line");

    const node = svg
      .append("g")
      .attr("stroke", "white")
      .attr("stroke-width", 0.5)
      .selectAll("circle")
      .data(nodes)
      .join("circle")
      .attr("r", 6)
      .attr("fill", (d) => d3.interpolateTurbo(d.id / nodes.length))
      .call(
        d3
          .drag()
          .on("start", (e, d) => {
            if (!e.active) sim.alphaTarget(0.3).restart();
            d.fx = d.x;
            d.fy = d.y;
          })
          .on("drag", (e, d) => {
            d.fx = e.x;
            d.fy = e.y;
          })
          .on("end", (e, d) => {
            if (!e.active) sim.alphaTarget(0);
            d.fx = null;
            d.fy = null;
          })
      );

    sim.on("tick", () => {
      link
        .attr("x1", (d) => d.source.x)
        .attr("y1", (d) => d.source.y)
        .attr("x2", (d) => d.target.x)
        .attr("y2", (d) => d.target.y);
      node.attr("cx", (d) => d.x).attr("cy", (d) => d.y);
    });

    return () => sim.stop();
  }, [nodes, links]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Rocket} title="Force‑Graph Playground" desc="Interactive network physics with D3" />
      </CardHeader>
      <CardContent>
        <div ref={ref} className="w-full" />
        <div className="flex gap-3 mt-3">
          <Button variant="secondary" onClick={() => setNodes(d3.range(20 + Math.floor(Math.random() * 20)).map((i) => ({ id: i })))}>
            Shuffle Nodes
          </Button>
          <Button variant="secondary" onClick={() => setLinks(d3.range(30 + Math.floor(Math.random() * 40)).map(() => ({ source: Math.floor(Math.random() * nodes.length), target: Math.floor(Math.random() * nodes.length) })))}>
            Regenerate Links
          </Button>
        </div>
      </CardContent>
    </Card>
  );
}

function ThreeCloud() {
  const mountRef = useRef(null);
  const [count, setCount] = useState(800);
  useEffect(() => {
    const mount = mountRef.current;
    if (!mount) return;

    const width = mount.clientWidth;
    const height = 320;

    const scene = new THREE.Scene();
    scene.fog = new THREE.Fog(0x00121f, 10, 60);
    const camera = new THREE.PerspectiveCamera(60, width / height, 0.1, 1000);
    camera.position.set(0, 0, 40);

    const renderer = new THREE.WebGLRenderer({ antialias: true, alpha: true });
    renderer.setSize(width, height);
    renderer.setPixelRatio(Math.min(window.devicePixelRatio, 2));
    mount.appendChild(renderer.domElement);

    const geometry = new THREE.BufferGeometry();
    const positions = new Float32Array(count * 3);
    for (let i = 0; i < count; i++) {
      const t = i / count * Math.PI * 6;
      const r = 10 + 8 * Math.sin(t * 0.7);
      positions[i * 3 + 0] = Math.cos(t) * r;
      positions[i * 3 + 1] = Math.sin(t * 0.9) * r * 0.6;
      positions[i * 3 + 2] = (Math.sin(t * 1.3) * 0.5 + Math.cos(t * 0.5) * 0.5) * 15;
    }
    geometry.setAttribute("position", new THREE.BufferAttribute(positions, 3));

    const material = new THREE.PointsMaterial({ size: 0.15 });
    const points = new THREE.Points(geometry, material);
    scene.add(points);

    const light = new THREE.PointLight(0xffffff, 1, 100);
    light.position.set(10, 10, 10);
    scene.add(light);

    let req;
    const animate = () => {
      points.rotation.y += 0.0025;
      points.rotation.x += 0.0015;
      renderer.render(scene, camera);
      req = requestAnimationFrame(animate);
    };
    animate();

    const onResize = () => {
      const w = mount.clientWidth;
      const h = 320;
      camera.aspect = w / h;
      camera.updateProjectionMatrix();
      renderer.setSize(w, h);
    };
    window.addEventListener("resize", onResize);

    return () => {
      cancelAnimationFrame(req);
      window.removeEventListener("resize", onResize);
      mount.removeChild(renderer.domElement);
      geometry.dispose();
      material.dispose();
      renderer.dispose();
    };
  }, [count]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Orbit} title="3D Spiral Point‑Cloud" desc="A particle nebula sculpted in math" />
      </CardHeader>
      <CardContent>
        <div ref={mountRef} className="w-full" style={{ height: 320 }} />
        <div className="mt-3">
          <div className="text-xs mb-2">Points: {count}</div>
          <Slider value={[count]} min={200} max={3000} step={50} onValueChange={(v) => setCount(v[0])} />
        </div>
      </CardContent>
    </Card>
  );
}

function RadialSkills() {
  const [levels, setLevels] = useState([80, 65, 72, 60, 90, 55]);
  const labels = ["Math", "Stats", "JS", "D3", "Three.js", "UX"];
  const data = labels.map((l, i) => ({ subject: l, A: levels[i], fullMark: 100 }));

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={RadarIcon} title="Radial Skills Radar" desc="Shape a profile as a polar flower" />
      </CardHeader>
      <CardContent className="space-y-4">
        <div className="h-64">
          <ResponsiveContainer width="100%" height="100%">
            <RadarChart data={data} cx="50%" cy="50%" outerRadius="80%">
              <PolarGrid />
              <PolarAngleAxis dataKey="subject" />
              <PolarRadiusAxis angle={30} domain={[0, 100]} />
              <Radar name="You" dataKey="A" strokeWidth={2} fillOpacity={0.2} />
              <Legend />
            </RadarChart>
          </ResponsiveContainer>
        </div>
        <div className="grid grid-cols-2 md:grid-cols-6 gap-3">
          {labels.map((l, i) => (
            <div key={l} className="text-center">
              <div className="text-xs mb-1">{l}</div>
              <Slider value={[levels[i]]} min={0} max={100} step={1} onValueChange={(v) => {
                const next = [...levels];
                next[i] = v[0];
                setLevels(next);
              }} />
            </div>
          ))}
        </div>
      </CardContent>
    </Card>
  );
}

function DonutBreakdown() {
  const [items, setItems] = useState([
    { name: "Exploration", value: 25 },
    { name: "Cleaning", value: 18 },
    { name: "Modeling", value: 28 },
    { name: "Visualization", value: 20 },
    { name: "Narrative", value: 9 },
  ]);
  const total = items.reduce((a, b) => a + b.value, 0);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Sparkles} title="Donut of Time" desc="Where your data‑science day goes" />
      </CardHeader>
      <CardContent className="grid grid-cols-1 md:grid-cols-2 gap-4 items-center">
        <div className="h-64">
          <ResponsiveContainer width="100%" height="100%">
            <PieChart>
              <Pie data={items} dataKey="value" nameKey="name" innerRadius={70} outerRadius={100}>
                {items.map((entry, index) => (
                  <Cell key={`c-${index}`} />
                ))}
              </Pie>
              <Tooltip />
              <Legend />
            </PieChart>
          </ResponsiveContainer>
        </div>
        <div className="space-y-2">
          {items.map((it, i) => (
            <div key={i} className="flex items-center gap-2">
              <Input
                value={it.name}
                onChange={(e) => {
                  const next = [...items];
                  next[i] = { ...next[i], name: e.target.value };
                  setItems(next);
                }}
              />
              <Input
                type="number"
                value={it.value}
                onChange={(e) => {
                  const next = [...items];
                  next[i] = { ...next[i], value: Number(e.target.value) };
                  setItems(next);
                }}
                className="w-24"
              />
            </div>
          ))}
          <div className="text-sm text-muted-foreground">Total: {total}</div>
        </div>
      </CardContent>
    </Card>
  );
}

function HeatmapNoise() {
  const ref = useRef(null);
  const [scale, setScale] = useState(30);

  useEffect(() => {
    const el = ref.current;
    if (!el) return;
    const width = el.clientWidth;
    const height = 280;
    el.innerHTML = "";

    const cols = 48;
    const rows = 24;
    const cellW = width / cols;
    const cellH = height / rows;

    const canvas = d3.select(el).append("canvas").attr("width", width).attr("height", height).style("width", "100%");
    const ctx = canvas.node().getContext("2d");

    const noise = (x, y) => {
      // simple layered noise
      let v = 0;
      let a = 1;
      let f = 1 / scale;
      for (let o = 0; o < 3; o++) {
        v += a * Math.sin(x * f + o * 3.1) * Math.cos(y * f + o * 1.7);
        a *= 0.5;
        f *= 2;
      }
      return (v + 2) / 4; // normalize 0..1 roughly
    };

    for (let r = 0; r < rows; r++) {
      for (let c = 0; c < cols; c++) {
        const x = c * cellW;
        const y = r * cellH;
        const v = noise(x, y);
        ctx.fillStyle = d3.interpolateTurbo(v);
        ctx.fillRect(x, y, cellW + 1, cellH + 1);
      }
    }
  }, [scale]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Hexagon} title="Procedural Heatmap" desc="Layered sinusoidal noise as data" />
      </CardHeader>
      <CardContent>
        <div ref={ref} className="w-full" style={{ height: 280 }} />
        <div className="mt-3">
          <div className="text-xs mb-2">Scale: {scale}</div>
          <Slider value={[scale]} min={10} max={80} step={1} onValueChange={(v) => setScale(v[0])} />
        </div>
      </CardContent>
    </Card>
  );
}

function P5Generative() {
  const hostRef = useRef(null);
  const [seed, setSeed] = useState(42);
  const [instance, setInstance] = useState(null);

  useEffect(() => {
    let cleanup = () => {};
    (async () => {
      const p5 = (await import("p5")).default;
      const sketch = (p) => {
        let zoff = 0;
        p.setup = () => {
          p.createCanvas(hostRef.current.clientWidth, 280);
          p.noiseSeed(seed);
        };
        p.windowResized = () => {
          p.resizeCanvas(hostRef.current.clientWidth, 280);
        };
        p.draw = () => {
          p.background(3, 12, 22);
          p.noFill();
          p.stroke(255, 255, 255, 120);
          const rows = 45;
          const cols = 80;
          const scale = 8;
          let yoff = 0;
          for (let y = 0; y < rows; y++) {
            let xoff = 0;
            p.beginShape();
            for (let x = 0; x < cols; x++) {
              const angle = p.noise(xoff, yoff, zoff) * Math.PI * 4;
              const vx = Math.cos(angle);
              const vy = Math.sin(angle);
              p.curveVertex(x * scale + vx * 8, y * scale + vy * 8);
              xoff += 0.05;
            }
            p.endShape();
            yoff += 0.05;
          }
          zoff += 0.0075;
        };
      };
      const inst = new p5(sketch, hostRef.current);
      setInstance(inst);
      cleanup = () => inst.remove();
    })();
    return () => cleanup();
  }, [seed]);

  return (
    <Card className="backdrop-blur-xl bg-white/5 border-white/10">
      <CardHeader>
        <SectionHeader icon={Sparkles} title="Generative Flow‑Field (p5.js)" desc="Noise‑driven ribbons that never repeat" />
      </CardHeader>
      <CardContent>
        <div ref={hostRef} className="w-full" />
        <div className="mt-3">
          <div className="text-xs mb-2">Seed: {seed}</div>
          <Slider value={[seed]} min={1} max={999} step={1} onValueChange={(v) => setSeed(v[0])} />
        </div>
      </CardContent>
    </Card>
  );
}

// ---------- Main Notebook ----------
export default function Notebook() {
  return (
    <div className="min-h-screen w-full bg-gradient-to-b from-slate-950 via-slate-900 to-slate-950 text-slate-100 p-6 md:p-10">
      <div className="max-w-7xl mx-auto">
        <header className="mb-8">
          <h1 className="text-3xl md:text-5xl font-extrabold tracking-tight flex items-center gap-3">
            <span className="inline-flex items-center justify-center p-2 rounded-2xl bg-white/5 backdrop-blur border border-white/10 shadow">
              <Atom className="w-6 h-6 md:w-8 md:h-8" />
            </span>
            JS Data‑Viz Notebook <span className="text-sky-400">(Unexpected Edition)</span>
          </h1>
          <p className="text-slate-300/80 mt-2 max-w-2xl">
            A wild, end‑to‑end playground of charts, math, and generative visuals. Tweak controls, break assumptions, and let the data sing.
          </p>
        </header>

        <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
          <TimeSeriesArcade />
          <CorrelationLab />
          <ClusterStudio />
          <ForceGraphPlayground />
          <ThreeCloud />
          <RadialSkills />
          <DonutBreakdown />
          <HeatmapNoise />
          <P5Generative />
        </div>

        <footer className="mt-10 text-xs text-slate-400/80">
          Built with React, Recharts, D3, Three.js, and p5.js • UI: Tailwind • Have an idea for a new cell? Tell me and I’ll add it.
        </footer>
      </div>
    </div>
  );
}


<IPython.core.display.Javascript object>